<a href="https://colab.research.google.com/github/omar80-coder/OCR-using-Azure-OCR-/blob/main/Azure_Focal_loss_LMv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"CUDA is available. GPU: {torch.cuda.get_device_name(0)}")
    # Set the default device to GPU
    torch.cuda.set_device(0)
else:
    print("CUDA is not available. Running on CPU.")


CUDA is not available. Running on CPU.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -q git+https://github.com/huggingface/transformers.git
# !pip install transformers datasets torch
#!pip install imgaug
#!pip install seqeval
#!pip install transformers[torch]
# !pip install accelerate -U
!pip install frontend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


In [ ]:
from transformers import LayoutLMv3Processor,LayoutLMv2ForTokenClassification,AutoProcessor
from PIL import Image, ImageDraw
import numpy as np
import tqdm as notebook_tqdm
import json
import pandas as pd
import os
#from pdf2image import convert_from_path
#import fitz  # PyMuPDF
import glob
import matplotlib.pyplot as plt
import cv2
from skimage import io
from PIL import Image, ImageDraw, ImageFont
import torch
from torch.utils.data import DataLoader, Dataset
#from datasets import load_metric
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [ ]:
def read_annotation_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

In [ ]:
# annotation_data= read_annotation_file("C:/Users/makni/Downloads/ieee/preprocessed_annotations/WRO-1.pdf.json")

# # annotation_data

In [ ]:
# from PIL import Image, ImageDraw
# import json

# def display_image_with_annotations(image_path, json_path, bbox_color='red'):
#     # Load the image and convert to RGB
#     image = Image.open(image_path).convert("RGB")
#     print(image.size)
#     draw = ImageDraw.Draw(image)

#     # Load the annotation data
#     with open(json_path, 'r') as file:
#         annotations = json.load(file)

#     # Iterate through the annotations and draw rectangles
#     for word, bbox in zip(annotations['words'], annotations['bboxes']):
#         draw.rectangle([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]], outline=bbox_color, width=2)
#         draw.text((bbox[0] + 2, bbox[1] + 2), word, fill='blue')

#     # Display the image
#     image.show()


# # Test the function
# annotation_path = "C:/Users/makni/Downloads/ieee/preprocessed_annotations/20231217_190133_merged-10.pdf.json"
# image_path = "C:/Users/makni/Downloads/ieee/preprocessed_images/20231217_190133_merged-10.pdf_1.png"
# display_image_with_annotations(image_path, annotation_path)  # Change bbox_color to any color you like


# Data Augmentation

In [ ]:
import albumentations as A
import cv2
import numpy as np
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
def get_train_transform():
    return A.Compose([
        A.MedianBlur(blur_limit=3, p=0.3),
        A.ColorJitter(p=0.3),
        A.PadIfNeeded(min_height=1000, min_width=700, border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255], p=1.0),
        A.Rotate(limit=5, p=1, interpolation=cv2.INTER_CUBIC),
    ], bbox_params=A.BboxParams(
        format='pascal_voc',
        label_fields=['labels'],
        min_area=0,
        min_visibility=0.1
    ))

In [ ]:
# !pip install albumentations

# LayoutLmv3

## Label Dictionaries:

In [ ]:
label2id = {
    'O': 0,
    'B-title': 1,
    'I-title': 2,
    'B-date': 3,
    'I-date': 4,
    'B-ieee': 5,
    'I-ieee': 6,
    'B-total': 7,
    'I-total': 8,
    'B-totalValue': 9,
    'I-totalValue': 10
}

# Create id2label by reversing label2id
id2label = {id: label for label, id in label2id.items()}

## Processor Initializationng

In [ ]:
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

## Custom Dataset Class:

In [ ]:
class CustomDataset:
    def __init__(self, annotation_dir, image_dir, annotation_files, transforms=None):
        self.annotation_dir = annotation_dir
        self.image_dir = image_dir
        self.annotation_files = annotation_files
        self.transforms = transforms

        # Load your annotations and image paths here
        self.data = []  # Store your data (image paths, annotations) in this list
        for annotation_file in self.annotation_files:
            # Load annotation from file
            annotation_path = os.path.join(self.annotation_dir, annotation_file)
            with open(annotation_path, 'r') as f:
                annotation = json.load(f)  # Assuming JSON format

            # Extract image path (adjust this based on your annotation structure)
            # Check if 'image_filename' exists, if not, try another key or handle the case accordingly.
            image_filename = annotation.get('image_filename')
            if image_filename is None:
                # Try a different key or handle the missing key
                image_filename = annotation.get('image_path', 'default_image.jpg')  # Replace 'image_path' and 'default_image.jpg' with appropriate values if needed
                print(f"Warning: 'image_filename' not found in {annotation_file}. Using '{image_filename}' instead.")

            image_path = os.path.join(self.image_dir, image_filename)

            self.data.append((image_path, annotation))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, annotation = self.data[idx]

        # Load image
        image = Image.open(image_path).convert('RGB')  # Adjust image loading as needed

        # Apply transformations if provided
        if self.transforms:
            image = self.transforms(image)

        # Return the image and corresponding annotation
        return image, annotation

In [ ]:
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

NameError: name 'LayoutLMv3Processor' is not defined

In [ ]:
import numpy as np
from torch.utils.data import DataLoader
from google.colab import drive
drive.mount('/content/drive')
annotation_dir = "/content/drive/MyDrive/finaly"
image_dir = "/content/drive/MyDrive/resized_hand"
annotation_files = [file for file in os.listdir(annotation_dir) if file.endswith('.json')]
annotation_files = [file for file in os.listdir(annotation_dir) if file.endswith('.json')]
np.random.shuffle(annotation_files)
split_idx = int(0.8 * len(annotation_files))
train_files = annotation_files[:split_idx]
test_files = annotation_files[split_idx:]
train_dataset = CustomDataset(annotation_dir, image_dir, annotation_files=train_files,transforms=get_train_transform()) #,transforms=get_train_transform()
test_dataset = CustomDataset(annotation_dir, image_dir, annotation_files=test_files)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# for i in range (1,156):
#     # print(i)
example = train_dataset[1]
# print(example)
# image= example["image"]
# print(image)
words = example["words"]
boxes = example["bbox"]
ner_tags = example["labels"]
    # ids= example["input_ids"]
    # masks= example["attention_mask"]
    # print("masks:", masks)
    # print("ids:", ids)

    # print(len(words),boxes.size() ,ner_tags.size() )
    # print("Bounding Boxes:", boxes.size())
    # # print("Bounding Boxes:", boxes)
    # print("NER Tags:", ner_tags.size())
    # print(boxes.shape)
    # print(ner_tags.shape)
    # print(ner_tags)
    # print(len(words))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/resized_hand/default_image.jpg'

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def custom_collate_fn(batch):
    max_len = max([len(item['input_ids']) for item in batch])
    input_ids = torch.stack([torch.cat([item['input_ids'], torch.tensor([processor.tokenizer.pad_token_id] * (max_len - len(item['input_ids'])), dtype=torch.long)]) for item in batch])
    attention_mask = torch.stack([torch.cat([item['attention_mask'], torch.tensor([0] * (max_len - len(item['attention_mask'])), dtype=torch.long)]) for item in batch])

    bbox_padded = []
    for item in batch:
        padding_length = max_len - item['bbox'].shape[0]
        padded_bbox = torch.cat([item['bbox'], torch.zeros(padding_length, 4, dtype=torch.int64)])
        bbox_padded.append(padded_bbox)
    bbox = torch.stack(bbox_padded)

    labels = torch.stack([torch.cat([item['labels'], torch.tensor([-100] * (max_len - len(item['labels'])), dtype=torch.long)]) for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'bbox': bbox,
        'labels': labels
    }

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=custom_collate_fn)

In [ ]:
for batch in train_loader:
    print({k: v.shape for k, v in batch.items()})
    break

print(train_loader)

{'input_ids': torch.Size([4, 512]), 'attention_mask': torch.Size([4, 512]), 'bbox': torch.Size([4, 512, 4]), 'labels': torch.Size([4, 512])}


# Define metrics

# Define the model

In [ ]:
from transformers import LayoutLMv3Processor,LayoutLMv2ForTokenClassification,AutoProcessor
model = LayoutLMv2ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv2-base-uncased",
    id2label=id2label,
    label2id=label2id)
#model = LayoutLMv2ForTokenClassification.from_pretrained("microsoft/layoutlmv2-base-uncased")

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/802M [00:00<?, ?B/s]

ImportError: 
LayoutLMv2Model requires the detectron2 library but it was not found in your environment. Checkout the instructions on the
installation page: https://github.com/facebookresearch/detectron2/blob/master/INSTALL.md and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:

!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install -q 'git+https://github.com/facebookresearch/detectron2.git'

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
try:
    import torch
    import torchvision
    import detectron2
    print("Torch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("Detectron2 is successfully installed.")
except ImportError as e:
    print("An error occurred:", e)


An error occurred: No module named 'detectron2'


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import numpy as np
import evaluate

# deprecated
# from datasets import load_metric
# metric = load_metric("seqeval")

metric = evaluate.load("seqeval")
label_list = list(model.config.id2label.values())


def compute_metrics(p, return_entity_level_metrics=False):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=1)
    if return_entity_level_metrics:
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [ ]:
# from transformers import LayoutLMv3FeatureExtractor, AutoTokenizer, LayoutLMv3Processor
# from transformers import LiltForTokenClassification
# # huggingface hub model id
# model_id = "SCUT-DLVCLab/lilt-roberta-en-base"
# # load model with correct number of labels and mapping
# model = LiltForTokenClassification.from_pretrained(
#     model_id, num_labels=11, label2id=label2id, id2label=id2label
# )

In [ ]:
# # Utilisez cette ligne pour créer l'optimiseur
# optimizer = AdamW(model.parameters(), lr=5e-3, no_deprecation_warning=True)
# # Move the model to the GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# Define TrainingArguments + Trainer


In [ ]:
from transformers import TrainingArguments, Trainer

# MLFlow

In [ ]:
!git clone https://github.com/akramLh005/PFA-mlflow.git
!cd PFA-mlflow


Cloning into 'PFA-mlflow'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 101 (delta 31), reused 65 (delta 8), pack-reused 0
Receiving objects: 100% (101/101), 20.20 KiB | 10.10 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [ ]:
#!pip install mlflow==2.11.1

In [ ]:
%env MLFLOW_TRACKING_URI=https://dagshub.com/akramLh005/PFA-mlflow.mlflow
%env MLFLOW_TRACKING_USERNAME=akramLh005
%env MLFLOW_TRACKING_PASSWORD=fabaf87fce39ef7e9bfdd406d6be4354ce2a25f4


env: MLFLOW_TRACKING_URI=https://dagshub.com/akramLh005/PFA-mlflow.mlflow
env: MLFLOW_TRACKING_USERNAME=akramLh005
env: MLFLOW_TRACKING_PASSWORD=fabaf87fce39ef7e9bfdd406d6be4354ce2a25f4


# Train

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from torch.nn.utils import clip_grad_norm_

def check_gradient(model):
    def hook(grad):
        if torch.isnan(grad).any() or torch.isinf(grad).any():
            print("NaN or Inf in gradients.")

    for p in model.parameters():
        p.register_hook(hook)



In [ ]:
# def calculate_class_counts(dataset):
#     # Initialiser un dictionnaire pour compter les occurrences de chaque classe
#     class_counts = {label: 0 for label in label2id.values()}

#     # Parcourir le jeu de données
#     for i in range(len(dataset)):
#         sample = dataset[i]
#         labels = sample['labels'].numpy()  # Assurez-vous que les labels sont sous forme de Numpy array pour comptage

#         # Compter chaque label
#         for label in labels:
#             if label != -100:  # Assurez-vous de ne pas compter les labels de padding
#                 class_counts[label] += 1

#     return class_counts

# # Appliquez cette fonction à votre jeu de données d'entraînement
# class_counts = calculate_class_counts(train_dataset)
# class_counts_list = [class_counts[i] for i in range(len(label2id))]  # Transformer en liste pour l'usage ultérieur

# print("Comptages de classes :", class_counts_list)


In [ ]:
# class_counts = [24727, 136, 39, 151, 50, 134, 307, 147, 102, 147, 30]  # Comptages fournis
# total_count = sum(class_counts)
# class_weights = [total_count / count for count in class_counts]

# # Convertir en tensor de poids pour PyTorch
# class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# total_count = sum(class_counts)
# class_weights = [total_count / (count + 1e-5) for count in class_counts]  # Small epsilon to avoid division by zero


# # Afficher les poids calculés pour chaque classe
# print("Poids pour chaque classe :", class_weights)
# from torch import nn
# device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
# model.to(device)


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 731.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=4bb965e52b480d16dd3feb76bf53f6b8f8e1094641180af137794bb3f6564e84
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import torch
torch.autograd.set_detect_anomaly(True)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


LayoutLMv2ForTokenClassification(
  (layoutlmv2): LayoutLMv2Model(
    (embeddings): LayoutLMv2Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (visual): LayoutLMv2VisualBackbone(
      (backbone): FPN(
        (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
        (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  

In [ ]:
# !pip install fvcore

In [ ]:
import torch
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification
from fvcore.nn import FlopCountAnalysis

# Assuming 'processor' and 'model' are initialized correctly somewhere in your code

image_path = "/content/drive/MyDrive/resized_images/20231217_190133_merged-13.pdf_1.png"
image = Image.open(image_path).convert("RGB")
example = train_dataset[1]
words = example["words"]
boxes = example["bbox"]
ner_tags = example["labels"]
input_ids = example["input_ids"]
attention_mask = example["attention_mask"]

# Ensure the encoding uses the processor correctly
encoding = processor(
    image,
    words,
    boxes=boxes,
    word_labels=ner_tags,
    return_tensors="pt"
)

# Move model and tensors to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {k: v.to(device) for k, v in encoding.items()}

model.eval()  # Set the model to evaluation mode

# Conduct FLOPs analysis - make sure to unpack the inputs correctly
flops = FlopCountAnalysis(model, (inputs['input_ids'], inputs['bbox'], inputs['attention_mask']))
#total_flops = flops.total()

#print(f"Total FLOPs: {total_flops}")


In [ ]:
import torch
import torch.nn as nn

# Total parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")


Total parameters: 200090443


In [ ]:
!pip install accelerate



In [ ]:
from transformers import TrainingArguments, Trainer, AdamW, get_scheduler
from transformers import EarlyStoppingCallback

from torch.optim import AdamW
import accelerate
print(accelerate.__version__)

from torch import nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = (1 - pt) ** self.gamma * BCE_loss

        if self.alpha is not None:
            if self.alpha.type() != inputs.data.type():
                self.alpha = self.alpha.type_as(inputs.data)
            at = self.alpha.gather(0, targets.data.view(-1))
            F_loss = at * F_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_func = FocalLoss(gamma=2.0)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss = self.loss_func(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

    def training_step(self, model, inputs):
        model.train()
        inputs = self._prepare_inputs(inputs)
        with self.autocast_smart_context_manager():
            loss = self.compute_loss(model, inputs)
        if self.args.gradient_accumulation_steps > 1:
            loss /= self.args.gradient_accumulation_steps
        self.accelerator.backward(loss)
        if self.args.max_grad_norm is not None:
            clip_grad_norm_(model.parameters(), self.args.max_grad_norm)
        return loss.detach()

training_args = TrainingArguments(
    output_dir="test_trainer",
    max_grad_norm=1.0,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    num_train_epochs=50,
    learning_rate=5e-5,
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=200,
    weight_decay=0.01,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    max_steps=2000,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)
# Mettre le modèle en mode CPU pour débogage
device = torch.device('cpu')
model.to(device)
# Assurez-vous que toutes les entrées sont également transférées sur CPU


trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor,
    data_collator=custom_collate_fn,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

weight_decay_rate = 0.01
optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=weight_decay_rate)
num_training_steps = training_args.max_steps
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps
)
training_args.lr_scheduler_type = 'linear'
training_args.lr_scheduler_num_warmup_steps = int(0.1 * num_training_steps)

trainer.train()

ModuleNotFoundError: No module named 'accelerate'

In [ ]:
import torch
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification
from fvcore.nn import FlopCountAnalysis

# Assuming 'processor' and 'model' are initialized correctly somewhere in your code
image_path = "resized_images/20231217_190133_merged-13.pdf_1.png"
image = Image.open(image_path).convert("RGB")
example = train_dataset[1]
words = example["words"]
boxes = example["bbox"]
ner_tags = example["labels"]
input_ids = example["input_ids"]
attention_mask = example["attention_mask"]

# Ensure the encoding uses the processor correctly
encoding = processor(
    image,
    words,
    boxes=boxes,
    word_labels=ner_tags,
    return_tensors="pt"
)

# Move model and tensors to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {k: v.to(device) for k, v in encoding.items()}

model.eval()  # Set the model to evaluation mode

# Conduct FLOPs analysis - make sure to unpack the inputs correctly
flops = FlopCountAnalysis(model, (inputs['input_ids'], inputs['bbox'], inputs['attention_mask']))
total_flops = flops.total()

print(f"Total FLOPs: {total_flops}")


C:\Users\makni\anaconda3\envs\PFA\lib\site-packages\transformers\modeling_utils.py:1051: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Unsupported operator aten::ne encountered 1 time(s)
Unsupported operator aten::cumsum encountered 1 time(s)
Unsupported operator aten::mul encountered 20 time(s)
Unsupported operator aten::add encountered 58 time(s)
Unsupported operator aten::embedding encountered 9 time(s)
Unsupported operator aten::add_ encountered 16 time(s)
Unsupported operator aten::sub encountered 17 time(s)
Unsupported operator aten::expand_as encountered 1 time(s)
Unsupported operator aten::rsub encountered 1 time(s)
Unsupported operator aten::abs encountered 3 time(s)
Unsupported operator aten::lt encountered 3 time(s)
Unsupported operator aten::div encountered 42 time(s)
Unsupported operator aten::log encountered 3 time(s)
Unsupported operator aten::min encountered 3 time(s)
Unsupported operator aten::where encou

Total FLOPs: 44832460800


In [ ]:
# from transformers import TrainingArguments, Trainer, AdamW, get_scheduler
# import torch
# import mlflow
# from transformers import EarlyStoppingCallback
# from torch.nn import functional as F

# mlflow.end_run()

# weight_decay_rate = 0.01

# training_args = TrainingArguments(
#     output_dir="test_trainer",
#     max_grad_norm=1.0,
#     evaluation_strategy="steps",
#     eval_steps=50,
#     save_strategy="steps",
#     save_steps=200,
#     save_total_limit=2,
#     num_train_epochs=50,
#     learning_rate=5e-4,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     fp16=True,
#     weight_decay=weight_decay_rate,
#     max_steps=1800,
#     load_best_model_at_end=True,
#     metric_for_best_model="f1",
#     greater_is_better=True,
# )
# class MLflowTrainer(Trainer):
#     def __init__(self, class_weights_tensor, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.class_weights_tensor = class_weights_tensor

#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.get('logits')
#         # Compute Focal Loss
#         loss = self.focal_loss(logits.view(-1, self.model.config.num_labels), labels.view(-1))
#         return (loss, outputs) if return_outputs else loss

#     def focal_loss(self, logits, labels, alpha=0.25, gamma=2, reduction='mean'):
#         cross_entropy = F.cross_entropy(logits, labels, reduction='none')
#         pt = torch.exp(-cross_entropy)
#         focal_loss = (alpha * (1 - pt) ** gamma * cross_entropy).mean()
#         return focal_loss

#     def training_step(self, model, inputs):
#         model.train()
#         inputs = self._prepare_inputs(inputs)
#         with self.autocast_smart_context_manager():
#             loss = self.compute_loss(model, inputs)
#         if self.args.gradient_accumulation_steps > 1:
#             loss /= self.args.gradient_accumulation_steps
#         self.accelerator.backward(loss)
#         if self.args.max_grad_norm is not None:
#             torch.nn.utils.clip_grad_norm_(model.parameters(), self.args.max_grad_norm)
#         logs = {"loss": loss.item()}
#         self.log(logs)
#         return loss.detach()

#     def log(self, logs):
#         super().log(logs)
#         for key, value in logs.items():
#             mlflow.log_metric(key, value, step=self.state.global_step)

#     def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
#         output = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
#         for key, value in output.items():
#             mlflow.log_metric(f"{metric_key_prefix}_{key}", value, step=self.state.global_step)

# trainer = MLflowTrainer(
#     class_weights_tensor=class_weights_tensor,
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     tokenizer=processor,
#     data_collator=custom_collate_fn,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
# )

# optimizer = AdamW(model.parameters(), lr=5e-4 , weight_decay=weight_decay_rate)
# num_training_steps = training_args.max_steps
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=int(0.1 * num_training_steps),
#     num_training_steps=num_training_steps
# )
# training_args.lr_scheduler_type = 'linear'
# training_args.lr_scheduler_num_warmup_steps = int(0.1 * num_training_steps)
# mlflow.start_run()
# trainer.train()
# mlflow.end_run()
